# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
\Large K=(X^{T}*Y)^{d}.
\end{equation}

In [2]:
import numpy as np

In [10]:
def build_kernel(data_set, kernel_type='linear', d=3, test_dataset=None):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        if test_dataset is None:
            kernel = kernel ** d
        else:
            kernel = np.dot(test_dataset, data_set.T) ** d
    return kernel

In [13]:
X = np.array([[0.5, 1],
              [0.2, 0.7],
              [0.1, 0.4]])

X

array([[0.5, 1. ],
       [0.2, 0.7],
       [0.1, 0.4]])

In [14]:
K = build_kernel(X, kernel_type='poly', d=1)
K

array([[1.25, 0.8 , 0.45],
       [0.8 , 0.53, 0.3 ],
       [0.45, 0.3 , 0.17]])

In [22]:
K = build_kernel(X, kernel_type='poly', d=3)
K

array([[1.953125, 0.512   , 0.091125],
       [0.512   , 0.148877, 0.027   ],
       [0.091125, 0.027   , 0.004913]])

In [26]:
test_set = np.array([[0.5, 0.1],
                     [0.5, 0.5],
                     [0.5, 1],
                     [0.5, 0.85],
                     [0.6, 0.4]])
K = build_kernel(X, kernel_type='poly', d=3, test_dataset=test_set)
K

array([[4.28750000e-02, 4.91300000e-03, 7.29000000e-04],
       [4.21875000e-01, 9.11250000e-02, 1.56250000e-02],
       [1.95312500e+00, 5.12000000e-01, 9.11250000e-02],
       [1.33100000e+00, 3.35702375e-01, 5.93190000e-02],
       [3.43000000e-01, 6.40000000e-02, 1.06480000e-02]])

row 2 col 0 -> data_set[0] and test_set[2]

those values are the same, therefore kernel value for them is the highest

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [229]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import cvxopt

In [230]:
# One vs One
def choose_set_for_label(data_set, labels, class_a, class_b):
    mask = np.logical_or(labels == class_a, labels == class_b)
    filtered_data = data_set[mask]
    filtered_labels = labels[mask]
    binary_labels = np.where(filtered_labels == class_a, 1, -1)
    return filtered_data, binary_labels

In [231]:
def get_labels_count(data_set):
    return len(data_set)

Use the code that we have implemented earlier:

In [232]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    
    objects_count = get_labels_count(train_data_set)
    
    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    # K = np.dot(test_data_set, support_vectors.T)
    # kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [241]:
iris = load_iris()
data_set = iris.data
labels = iris.target
print(len(data_set))
print(np.unique(labels))
X_train, X_test, y_train, y_test = train_test_split(data_set, labels, test_size=0.2, random_state=42)

150
[0 1 2]


In [242]:
data_set

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [243]:
# modify this part 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ONE VS ONE
train_01, labels_train_01 = choose_set_for_label(X_train_scaled, y_train, 0, 1)
train_02, labels_train_02 = choose_set_for_label(X_train_scaled, y_train, 0, 2)
train_12, labels_train_12 = choose_set_for_label(X_train_scaled, y_train, 1, 2)

test_01, labels_test_01 = choose_set_for_label(X_test_scaled, y_test, 0, 1)
test_02, labels_test_02 = choose_set_for_label(X_test_scaled, y_test, 0, 2)
test_12, labels_test_12 = choose_set_for_label(X_test_scaled, y_test, 1, 2)

clf0 = train(train_01, labels_train_01, kernel_type='rbf')
clf1 = train(train_02, labels_train_02, kernel_type='rbf')
clf2 = train(train_12, labels_train_12, kernel_type='rbf')

     pcost       dcost       gap    pres   dres
 0:  1.1559e+02 -1.4213e+03  3e+03  2e-01  3e-15
 1:  6.8399e+01 -1.4534e+02  2e+02  6e-03  3e-15
 2:  8.0750e+00 -2.0097e+01  3e+01  2e-15  2e-15
 3: -8.8705e-01 -4.6985e+00  4e+00  7e-16  9e-16
 4: -1.6927e+00 -2.5407e+00  8e-01  2e-16  3e-16
 5: -1.8888e+00 -2.1881e+00  3e-01  2e-16  2e-16
 6: -1.9866e+00 -2.0881e+00  1e-01  4e-16  3e-16
 7: -2.0157e+00 -2.0418e+00  3e-02  6e-16  3e-16
 8: -2.0244e+00 -2.0301e+00  6e-03  2e-16  3e-16
 9: -2.0266e+00 -2.0273e+00  7e-04  2e-16  3e-16
10: -2.0269e+00 -2.0269e+00  9e-06  6e-16  3e-16
11: -2.0269e+00 -2.0269e+00  9e-08  4e-16  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.2966e+02 -1.4503e+03  3e+03  2e-01  3e-15
 1:  7.1384e+01 -1.6424e+02  3e+02  8e-03  2e-15
 2:  8.2839e+00 -2.2400e+01  3e+01  2e-16  2e-15
 3: -1.3102e+00 -5.3941e+00  4e+00  3e-16  8e-16
 4: -2.1991e+00 -3.0900e+00  9e-01  2e-16  4e-16
 5: -2.3973e+00 -2.6400e+00  2e-01  2e-16  3e-1

In [244]:
X_train_scaled

array([[-1.47393679,  1.20365799, -1.56253475, -1.31260282],
       [-0.13307079,  2.99237573, -1.27600637, -1.04563275],
       [ 1.08589829,  0.08570939,  0.38585821,  0.28921757],
       [-1.23014297,  0.75647855, -1.2187007 , -1.31260282],
       [-1.7177306 ,  0.30929911, -1.39061772, -1.31260282],
       [ 0.59831066, -1.25582892,  0.72969227,  0.95664273],
       [ 0.72020757,  0.30929911,  0.44316389,  0.4227026 ],
       [-0.74255534,  0.98006827, -1.27600637, -1.31260282],
       [-0.98634915,  1.20365799, -1.33331205, -1.31260282],
       [-0.74255534,  2.32160658, -1.27600637, -1.44608785],
       [-0.01117388, -0.80864948,  0.78699794,  0.95664273],
       [ 0.23261993,  0.75647855,  0.44316389,  0.55618763],
       [ 1.08589829,  0.08570939,  0.55777524,  0.4227026 ],
       [-0.49876152,  1.87442714, -1.39061772, -1.04563275],
       [-0.49876152,  1.4272477 , -1.27600637, -1.31260282],
       [-0.37686461, -1.47941864, -0.01528151, -0.24472256],
       [ 0.59831066, -0.

In [245]:
test_datasets = [test_01, test_02, test_12]
train_datasets = [train_01, train_02, train_12]
test_labels = [labels_test_01, labels_test_02, labels_test_12]
print("Accuracy for each classifier")
for i, clf in enumerate([clf0, clf1, clf2]):
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = clf
    predicted = list(classify_rbf(test_datasets[i], train_datasets[i], lambdas, targets, b, vector_number, support_vectors, support_vectors_id).astype(int))
    print(accuracy_score(predicted, test_labels[i]))

Accuracy for each classifier
1.0
0.8571428571428571
0.55


In [246]:
print(labels_test_01)
print(labels_test_02)
print(labels_test_12)

[-1  1 -1 -1  1 -1 -1 -1  1  1  1  1 -1 -1 -1  1  1  1  1]
[ 1 -1  1 -1 -1  1  1  1  1 -1 -1  1 -1  1 -1 -1 -1 -1 -1  1  1]
[ 1 -1  1  1  1 -1  1  1 -1  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1]


In [247]:
# Predictions
from sklearn.metrics import accuracy_score
from collections import Counter

predictions = []
for i, clf in enumerate([clf0, clf1, clf2]):
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = clf
    predicted = list(classify_rbf(X_test_scaled, X_train_scaled, lambdas, targets, b, vector_number, support_vectors, support_vectors_id).astype(int))
    predictions.append(predicted)

# voting
final_predictions = []
for i in range(len(predictions[0])):
    votes = []
    for j in range(len(predictions)):
        if j == 0:
            if predictions[j][i] == 1:
                votes.append(0)
            else:
                votes.append(1)
        elif j == 1:
            if predictions[j][i] == 1:
                votes.append(0)
            else:
                votes.append(2)
        else:
            if predictions[j][i] == 1:
                votes.append(1)
            else:
                votes.append(2)
    final_class = Counter(votes).most_common(1)[0][0]
    final_predictions.append(final_class)

print(accuracy_score(y_test, final_predictions))

0.7666666666666667


In [248]:
for i in range(3):
    print(np.ravel(predictions[i]))

[-1  1 -1 -1 -1  1 -1 -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1  1 -1  1 -1
 -1 -1 -1 -1  1  1]
[-1  1 -1 -1 -1  1  1 -1 -1 -1 -1  1  1  1  1 -1 -1  1 -1 -1  1 -1  1 -1
 -1 -1 -1 -1  1  1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1]
